In [9]:
import numpy as np
import jax
import jax.numpy as jnp
import random
import pandas as pd
import time

In [10]:
def gen_initial_conditions(n):
    matriz = np.zeros((n, n))
    for i in range(n):
        matriz[i, i] = (i+1)/2

    gen_x = jnp.array([1.0 - i/n for i in range(n)])
    return matriz, gen_x


In [11]:
def f_rosenbrock(x):
    x = jnp.array(x)
    total = 0.0
    N = x.shape[0]

    for i in range(0, N-1, 2):
        total += 100*(x[i+1] - x[i]**2)**2 + (1 - x[i])**2

    return total

In [12]:
calculate_grad_rosembrock = jax.grad(f_rosenbrock)
calculate_hess_rosembrock = jax.hessian(f_rosenbrock)

In [13]:
def newton_method(iterations, x, tol=1e-3):
    x_hist = [jnp.array(x[0])]
    for k in range(iterations):
        xk = x_hist[-1]
        gk = calculate_grad_rosembrock(xk)
        Hk = calculate_hess_rosembrock(xk)
        grad_norm = float(jnp.linalg.norm(gk))
        print(f"[Newton] Iter {k} | ||grad|| = {grad_norm:.3e}")
        if grad_norm < tol:
            break
        try:
            p = jnp.linalg.solve(Hk, -gk)
        except Exception as e:
            print(f"[Newton] Hessiana singular na iter {k}: {e}")
            break
        x_next = xk + p
        x_hist.append(x_next)
    return x_hist


In [14]:
import time
import jax.numpy as jnp

def newton_method_with_max_time(iterations, x, tol=1e-3, max_time=600):
    """
    Método de Newton:
    Para ao atingir ||grad|| < tol ou ao exceder max_time segundos.
    """
    x_hist = [jnp.array(x[0])]
    start_time = time.time()

    for k in range(iterations):

        # Checar tempo máximo
        elapsed = time.time() - start_time
        if elapsed > max_time:
            print(f"[Parada] Tempo máximo de {max_time}s excedido (tempo = {elapsed:.2f}s)")
            Z=2
            return Z


        xk = x_hist[-1]
        gk = calculate_grad_rosembrock(xk)
        Hk = calculate_hess_rosembrock(xk)
        grad_norm = float(jnp.linalg.norm(gk))

        print(f"[Iter {k}] ||grad|| = {grad_norm:.6e}  (tempo = {elapsed:.2f}s)")

        # Critério de parada pelo gradiente
        if grad_norm < tol:
            break

        # Resolver Hessiana * p = -grad
        try:
            p = jnp.linalg.solve(Hk, -gk)
        except:
            break

        x_next = xk + p
        x_hist.append(x_next)

    return x_hist, Z


In [ ]:

def newton_method_with_metrics(iterations, x,to, max_timel):
    start = time.time()

    x_hist = newton_method(iterations, x)

    elapsed = time.time() - start
    f_vals = [float(f_rosenbrock(xi)) for xi in x_hist]
    grad_vals = [float(jnp.linalg.norm(calculate_grad_rosembrock(xi))) for xi in x_hist]

    return {
        "x": x_hist,
        "iterations": len(x_hist) - 1,
        "time": elapsed,
        "final_f": f_vals[-1],
        "final_grad_norm": grad_vals[-1],
        "f_values": f_vals,
        "grad_norms": grad_vals
    }

In [16]:
N_values = [1, 10, 100, 1000]
λ_values = [1.0]

results = []

for n in N_values:
    for λ in λ_values:
        
        A, x0 = gen_initial_conditions(n)
        x = [x0]

        metrics = newton_method_with_metrics(
            iterations=10000,
            x=x,
            tol=1e-3,
            max_time=800
        )

        results.append({
            'N': n,
            'λ': λ,
            'Iterações': metrics['iterations'],
            'Tempo (s)': metrics['time'],
            'f(x_final)': metrics['final_f'],
            '||∇f(x_final)||': metrics['final_grad_norm'],
            'timeout': metrics['timeout'],
            'metrics': metrics
        })

        # Se deu timeout, interrompe o loop dessa dimensão
        if metrics['timeout']:
            print(f"[Aviso] Timeout atingido para N={n}. Interrompendo...")
            break


df = pd.DataFrame([{k: v for k, v in r.items() if k != 'metrics'} for r in results])

print("="*70)
print("RESULTADOS - Método de Newton (Rosenbrock)")
print("="*70)
print(df.to_string(index=False))
print("="*70)


TypeError: newton_method_with_metrics() got an unexpected keyword argument 'tol'